Pull data from file.

In [39]:
import pandas as pd

heart_data_2022_nan = pd.read_csv("./Archive/2022/heart_2022_with_nans.csv")

Pick data we cant to use.

In [40]:
df_model = heart_data_2022_nan[['HadHeartAttack', 'Sex', 'AgeCategory', 'BMI', 'SmokerStatus', 'SleepHours', 'AlcoholDrinkers', 'HadStroke']].dropna()

print(df_model.head())

  HadHeartAttack     Sex      AgeCategory    BMI   SmokerStatus  SleepHours  \
1             No  Female  Age 80 or older  26.57   Never smoked         6.0   
2             No  Female     Age 55 to 59  25.61   Never smoked         5.0   
4             No  Female     Age 40 to 44  21.77   Never smoked         9.0   
5            Yes    Male  Age 80 or older  26.08   Never smoked         7.0   
6             No  Female  Age 80 or older  22.96  Former smoker         7.0   

  AlcoholDrinkers HadStroke  
1              No        No  
2              No        No  
4             Yes        No  
5              No       Yes  
6             Yes        No  


Now One-hot Encode

In [45]:
df_encoded = pd.get_dummies(df_model, drop_first=True)

print(df_encoded.head())

     BMI  SleepHours  HadHeartAttack_Yes  Sex_Male  AgeCategory_Age 25 to 29  \
1  26.57         6.0               False     False                     False   
2  25.61         5.0               False     False                     False   
4  21.77         9.0               False     False                     False   
5  26.08         7.0                True      True                     False   
6  22.96         7.0               False     False                     False   

   AgeCategory_Age 30 to 34  AgeCategory_Age 35 to 39  \
1                     False                     False   
2                     False                     False   
4                     False                     False   
5                     False                     False   
6                     False                     False   

   AgeCategory_Age 40 to 44  AgeCategory_Age 45 to 49  \
1                     False                     False   
2                     False                     False   
4    

Now Scaling

In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(df_encoded)
df_scaled = pd.DataFrame(scaled_values, columns=df_encoded.columns)
print(df_scaled.head())

        BMI  SleepHours  HadHeartAttack_Yes  Sex_Male  \
0  0.166058    0.217391                 0.0       0.0   
1  0.155102    0.173913                 0.0       0.0   
2  0.111276    0.347826                 0.0       0.0   
3  0.160466    0.260870                 1.0       1.0   
4  0.124857    0.260870                 0.0       0.0   

   AgeCategory_Age 25 to 29  AgeCategory_Age 30 to 34  \
0                       0.0                       0.0   
1                       0.0                       0.0   
2                       0.0                       0.0   
3                       0.0                       0.0   
4                       0.0                       0.0   

   AgeCategory_Age 35 to 39  AgeCategory_Age 40 to 44  \
0                       0.0                       0.0   
1                       0.0                       0.0   
2                       0.0                       1.0   
3                       0.0                       0.0   
4                       0.0  

Throw data at Model

Star with Linear Regression

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report

X = df_scaled[['BMI', 'SleepHours', 'Sex_Male', 'AgeCategory_Age 25 to 29', 'AgeCategory_Age 30 to 34', \
    'AgeCategory_Age 35 to 39',  'AgeCategory_Age 40 to 44', 'AgeCategory_Age 45 to 49', 'AgeCategory_Age 50 to 54', \
    'AlcoholDrinkers_Yes', 'HadStroke_Yes']]
y = df_scaled['HadHeartAttack_Yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the linear regression model
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

# Make predictions
y_pred = linear_reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Linear Regression Mean Squared Error: {mse}")

Linear Regression Mean Squared Error: 0.0523181972018557


Evaluation & Insights

In [58]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

#cm = confusion_matrix(y_test, y_pred)
#sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
#plt.xlabel("Predicted")
#plt.ylabel("Actual")
#plt.title("Confusion Matrix")
#plt.show()

# Evaluate the model with confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["High", "Not High"], yticklabels=["High", "Not High"])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Logistic Regression Confusion Matrix (High vs. Not High)")
plt.show()

ValueError: Classification metrics can't handle a mix of binary and continuous targets